# Additional Readings
You can read the full description of project here:
https://medium.com/@Markeko/speech-emotion-recognition-with-convolutional-neural-network-ae5406a1c0f7

# Import libraries

In [1]:
#Import data packages
import os
import sys
import glob
import numpy as np
import pandas as pd

#Import audio packages
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.io.wavfile
import sys

#Import plotting packages
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import matplotlib.pyplot as plt
import seaborn as sns

#Import Keras & Tensorflow packages
import keras
import tensorflow as tf
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

# Read Files

There are two databases that were used for this project:
- RAVDESS: The RAVDESS file contains a unique filename that consists in a 7-part numerical identifier.
- TESS: The TESS file contains a unique letter at beginning of file name to identify the emotion.

## Read RAVDESS Dataset

In [2]:
#Build list of files
rawdata_list = os.listdir('RawData/')

In [3]:
#Review list of files
print(rawdata_list)

['03-01-01-01-01-01-01.wav', '03-01-01-01-01-01-02.wav', '03-01-01-01-01-01-03.wav', '03-01-01-01-01-01-04.wav', '03-01-01-01-01-01-05.wav', '03-01-01-01-01-01-06.wav', '03-01-01-01-01-01-07.wav', '03-01-01-01-01-01-08.wav', '03-01-01-01-01-01-09.wav', '03-01-01-01-01-01-10.wav', '03-01-01-01-01-01-11.wav', '03-01-01-01-01-01-12.wav', '03-01-01-01-01-01-13.wav', '03-01-01-01-01-01-14.wav', '03-01-01-01-01-01-15.wav', '03-01-01-01-01-01-16.wav', '03-01-01-01-01-01-17.wav', '03-01-01-01-01-01-18.wav', '03-01-01-01-01-01-19.wav', '03-01-01-01-01-01-20.wav', '03-01-01-01-01-01-21.wav', '03-01-01-01-01-01-22.wav', '03-01-01-01-01-01-23.wav', '03-01-01-01-01-01-24.wav', '03-01-01-01-01-02-01.wav', '03-01-01-01-01-02-02.wav', '03-01-01-01-01-02-03.wav', '03-01-01-01-01-02-04.wav', '03-01-01-01-01-02-05.wav', '03-01-01-01-01-02-06.wav', '03-01-01-01-01-02-07.wav', '03-01-01-01-01-02-08.wav', '03-01-01-01-01-02-09.wav', '03-01-01-01-01-02-10.wav', '03-01-01-01-01-02-11.wav', '03-01-01-01-01-02-

## LIbrosa & MFCC configuration
In order to analyze and standardize how each audio file feature was built, the following configurations were determined:

In [4]:
#sample feature
#librosa.core.load(path, sr=22050, mono=True, offset=0.0, duration=None, dtype=<class 'numpy.float32'>, res_type='kaiser_best')
res_type_s = 'kaiser_best'
duration_s = None
sample_rate_s = 22050
offset_s = 0.5

#Mfcc
#librosa.feature.mfcc(y=None, sr=22050, S=None, n_mfcc=20, dct_type=2, norm='ortho', lifter=0, **kwargs)
mfcc_sample_rate = 22050
n_mfcc = 40
axis_mfcc = 1

### RAVDESS get emotion features

File naming convention

Each of the 7356 RAVDESS files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 02-01-06-01-02-01-12.mp4). These identifiers define the stimulus characteristics: 

Filename identifiers 

- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
- Vocal channel (01 = speech, 02 = song).
- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
- Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
- Repetition (01 = 1st repetition, 02 = 2nd repetition).
- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

Filename example: 02-01-06-01-02-01-12.mp4 
- Video-only (02)
- Speech (01)
- Fearful (06)
- Normal intensity (01)
- Statement "dogs" (02)
- 1st Repetition (01)
- 12th Actor (12)
- Female, as the actor ID number is even.

In [5]:
#Build list with target variables for each file
feeling_list=[]

#Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fear, 07 = disgust, 08 = surprised) 

for emotion_path in rawdata_list:
    if emotion_path.split('-')[2] == '01':
        feeling_list.append("neutral")
    elif emotion_path.split('-')[2] == '02':
        feeling_list.append("calm")
    elif emotion_path.split('-')[2] == '03':
        feeling_list.append("happy")
    elif emotion_path.split('-')[2] == '04':
        feeling_list.append("sad")
    elif emotion_path.split('-')[2] == '05':
        feeling_list.append("angry")
    elif emotion_path.split('-')[2] == '06':
        feeling_list.append("fear")
    elif emotion_path.split('-')[2] == '07':
        feeling_list.append("disgust")
    elif emotion_path.split('-')[2] == '08':
        feeling_list.append("surprised")
    else:
        feeling_list.append("unknown")

In [6]:
#Check list
feeling_list

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ne

In [7]:
#Turn list into dataframe
labels = pd.DataFrame(feeling_list)

In [8]:
#Check shape
labels.shape

(1440, 1)

In [9]:
#Change index name to "emotion"
labels = labels.rename({0: 'emotion'}, axis=1)

In [10]:
labels.shape

(1440, 1)

In [11]:
#Count the number of files per emotion
labels_total = pd.DataFrame(labels.groupby(['emotion']).size())
labels_total

,0
emotion,
angry,192
calm,192
disgust,192
fear,192
happy,192
neutral,96
sad,192
surprised,192


### RAVDESS get audio features with librosa library

In [12]:

rawdata_ravdess = pd.DataFrame(columns=['feature'])
bookmark=0

for y in rawdata_list:
    #Change to kaiser_best & 22050 kHz
    #sr > target sampling rate
    #offset=0.5
    X, sample_rate = librosa.load('RawData/'+y, 
                                  res_type = res_type_s,
                                  duration = duration_s,
                                  sr = sample_rate_s,
                                  offset = offset_s)
    sample_rate = np.array(sample_rate)
    
    #Get MFCCs from each file
    mfccs = librosa.feature.mfcc(   y=X, 
                                    sr = mfcc_sample_rate, 
                                    n_mfcc = n_mfcc)
    
    #Calculate mean of MFCCs
    mfccs_mean = np.mean(    mfccs, 
                             axis = axis_mfcc)
    feature = mfccs_mean
    
    #Add MFCCs feature results to list
    rawdata_ravdess.loc[bookmark] = [feature]
    bookmark=bookmark+1   

In [13]:
#Verify data results
rawdata_ravdess.shape

(1440, 1)

In [14]:
#Verify that there are no null values
rawdata_ravdess.isnull().values.any()

False

In [15]:
# See array sample of features
rawdata_ravdess

,feature
0,"[-671.3947, 66.76209, -0.8031736, 16.397806, 7..."
1,"[-624.36914, 64.56482, -12.068572, 11.360991, ..."
2,"[-586.3834, 67.529655, -4.9390116, 13.85341, 3..."
3,"[-664.51685, 52.24366, -10.568827, 12.448711, ..."
4,"[-689.68176, 79.54792, 6.3848214, 17.58376, 10..."
...,...
1435,"[-562.17584, 40.83896, -22.902687, 7.2095323, ..."
1436,"[-488.2176, 54.026115, -5.6680098, 14.405636, ..."
1437,"[-509.09995, 31.333313, -18.702072, -3.8738506..."
1438,"[-499.69315, 41.940163, -15.661548, -0.1003641..."


In [16]:
#Turn array into dataframe
rawdata_ravdess_final = pd.DataFrame(rawdata_ravdess['feature'].values.tolist())

In [17]:
#Analyze new dataframe shape
rawdata_ravdess_final.shape

(1440, 40)

In [18]:
# Check data sample
rawdata_ravdess_final.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-671.394714,66.762093,-0.803174,16.397806,7.514786,2.309502,-5.473440,-1.983500,-14.529568,-1.763067,...,-2.734973,-2.177600,-3.423326,-1.972813,-3.967898,-1.697054,-1.329119,-2.123123,-4.130067,-1.843433
1,-624.369141,64.564819,-12.068572,11.360991,-6.662220,-2.700722,-7.720459,-8.432280,-6.146567,0.402561,...,-0.008967,3.946117,2.053208,5.126789,2.357050,3.340448,-1.931583,1.640402,-0.995546,0.893389
2,-586.383423,67.529655,-4.939012,13.853410,3.375603,-1.342491,-15.607982,-6.360179,-5.104679,-8.572944,...,-2.091636,-1.549652,-1.892016,1.164338,-0.055426,-1.253459,-3.225636,-2.361140,-1.318202,2.959448
3,-664.516846,52.243660,-10.568827,12.448711,-6.724777,-4.079839,-9.436715,-13.167788,-10.333027,-2.645067,...,-1.477941,1.114804,-2.729333,2.110939,2.055382,3.083011,1.030954,1.056760,4.401068,5.721843
4,-689.681763,79.547920,6.384821,17.583759,10.556285,0.527172,-1.631915,-2.790388,-2.794857,2.297231,...,-0.924785,0.894624,-0.286552,0.833501,-0.318295,2.581150,-0.990544,-1.715069,-2.530386,0.056745


## Ravdess join features and target

In [19]:
#Join labels with features
newdf_ravdess = pd.concat([rawdata_ravdess_final,labels], axis=1)

In [20]:
#Rename dataframe
newdf_ravdess = newdf_ravdess.rename(index=str, columns={"0": "label"})

In [21]:
#Analyze dataframe shape
newdf_ravdess.shape

(1440, 41)

In [22]:
#Anayze dataframe sample
newdf_ravdess.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-671.394714,66.762093,-0.803174,16.397806,7.514786,2.309502,-5.473440,-1.983500,-14.529568,-1.763067,...,-2.177600,-3.423326,-1.972813,-3.967898,-1.697054,-1.329119,-2.123123,-4.130067,-1.843433,neutral
1,-624.369141,64.564819,-12.068572,11.360991,-6.662220,-2.700722,-7.720459,-8.432280,-6.146567,0.402561,...,3.946117,2.053208,5.126789,2.357050,3.340448,-1.931583,1.640402,-0.995546,0.893389,neutral
2,-586.383423,67.529655,-4.939012,13.853410,3.375603,-1.342491,-15.607982,-6.360179,-5.104679,-8.572944,...,-1.549652,-1.892016,1.164338,-0.055426,-1.253459,-3.225636,-2.361140,-1.318202,2.959448,neutral
3,-664.516846,52.243660,-10.568827,12.448711,-6.724777,-4.079839,-9.436715,-13.167788,-10.333027,-2.645067,...,1.114804,-2.729333,2.110939,2.055382,3.083011,1.030954,1.056760,4.401068,5.721843,neutral
4,-689.681763,79.547920,6.384821,17.583759,10.556285,0.527172,-1.631915,-2.790388,-2.794857,2.297231,...,0.894624,-0.286552,0.833501,-0.318295,2.581150,-0.990544,-1.715069,-2.530386,0.056745,neutral


In [23]:
#Datafram drop Nan values
newdf_ravdess.dropna(inplace=True)

In [24]:
from sklearn.utils import shuffle

#Shuffle dataframe
newdf_ravdess = shuffle(newdf_ravdess)
newdf_ravdess.head(10)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
9,-544.557678,60.222256,-3.353786,5.997199,2.195110,-3.345102,-11.411964,-7.863065,-7.738362,-0.604156,...,4.648522,5.395966,4.909775,5.915462,3.823320,1.722570,2.944994,0.525525,4.867732,neutral
458,-380.761108,52.812840,-12.213782,-1.671794,-5.027451,3.621909,-20.101509,-8.210300,-11.666789,-12.271439,...,0.877870,0.522170,4.972847,3.117938,4.291908,0.009138,0.469729,-2.325377,3.876662,happy
490,-668.349365,62.966152,14.921709,22.523623,11.318936,8.434793,0.933675,5.188271,-6.645576,2.771111,...,2.112148,0.294668,0.178667,-1.513757,-1.903754,-2.710942,-0.492939,-3.375104,-0.474419,sad
210,-652.636841,76.974335,12.686167,23.314371,7.070840,13.493074,-6.541290,-2.230219,-12.407928,-2.440194,...,-0.519305,-1.395967,2.385193,-1.998615,2.372705,-4.737188,0.767688,-3.771387,1.944862,calm
1384,-464.399536,45.754799,0.021852,14.143663,2.155463,9.609376,-0.218632,1.176242,-5.241661,5.230917,...,5.429664,2.029474,4.688261,0.942630,3.916699,2.372335,4.352333,1.602945,4.243527,surprised
156,-697.714661,80.303444,12.071409,15.261730,10.391933,6.305645,-1.662077,-5.108406,-8.237923,3.708027,...,-0.944229,-1.111994,0.713900,-0.869575,2.525424,-3.844625,-0.627082,-4.015927,-0.752813,calm
553,-655.503113,52.820831,-7.789691,9.070624,-4.870082,-3.096890,-9.382129,-10.186951,-7.768711,0.429273,...,12.735394,12.867529,12.435796,8.794301,6.419610,-1.993194,0.243939,0.322978,3.465655,sad
928,-496.441589,64.705734,-10.641956,13.125334,10.812528,6.476342,0.682608,-6.713028,-13.808793,8.824391,...,0.595457,1.542647,3.395404,0.591234,0.543090,1.134280,1.614634,0.300022,2.420401,fear
781,-355.467163,37.354416,-37.379475,-1.219587,-26.438776,-16.093231,-6.834251,-14.464345,-9.492141,-3.310290,...,0.844329,-1.537415,1.593976,-1.172927,1.508093,-0.710842,-0.380396,-1.789901,2.421494,angry
511,-709.939453,78.086952,7.368532,25.333670,4.129424,4.473402,1.661272,-4.555212,-1.808529,-1.904574,...,3.498953,2.052567,8.376937,9.241560,12.590361,7.223436,5.173040,2.806487,3.234638,sad


In [25]:
#Verify that there are no null values
newdf_ravdess.isnull().values.any()

False

In [26]:
# Check dataframe sample
newdf_ravdess.head(5)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
9,-544.557678,60.222256,-3.353786,5.997199,2.195110,-3.345102,-11.411964,-7.863065,-7.738362,-0.604156,...,4.648522,5.395966,4.909775,5.915462,3.823320,1.722570,2.944994,0.525525,4.867732,neutral
458,-380.761108,52.812840,-12.213782,-1.671794,-5.027451,3.621909,-20.101509,-8.210300,-11.666789,-12.271439,...,0.877870,0.522170,4.972847,3.117938,4.291908,0.009138,0.469729,-2.325377,3.876662,happy
490,-668.349365,62.966152,14.921709,22.523623,11.318936,8.434793,0.933675,5.188271,-6.645576,2.771111,...,2.112148,0.294668,0.178667,-1.513757,-1.903754,-2.710942,-0.492939,-3.375104,-0.474419,sad
210,-652.636841,76.974335,12.686167,23.314371,7.070840,13.493074,-6.541290,-2.230219,-12.407928,-2.440194,...,-0.519305,-1.395967,2.385193,-1.998615,2.372705,-4.737188,0.767688,-3.771387,1.944862,calm
1384,-464.399536,45.754799,0.021852,14.143663,2.155463,9.609376,-0.218632,1.176242,-5.241661,5.230917,...,5.429664,2.029474,4.688261,0.942630,3.916699,2.372335,4.352333,1.602945,4.243527,surprised


In [27]:
#Analyz shape of dataframe
newdf_ravdess.shape

(1440, 41)

In [28]:
# see number of emotions
newdf_ravdess[newdf_ravdess.columns[-1]].nunique()

8

## Read TESS Dataset

In [31]:
# Build list of audio files
raw_data_tess_path = r'C:\Users\daveb\OneDrive\Documents\GitHub\speech_emotion_analyzer_project\RawData2'


folder_list_tess = os.listdir(raw_data_tess_path)

tess_list = []

for folder in folder_list_tess:
    #folder_path = raw_data_tess_path+folder+"\\"
    folder_path = raw_data_tess_path+"\\"
    os.chdir(folder_path)
    for file in glob.glob("*.wav"):
        tess_list.append(folder_path+file)

#Check results
tess_list[:10]

['C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_angry.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_disgust.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_fear.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_happy.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_neutral.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_ps.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_back_sad.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_bar_angry.wav',
 'C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\speech_emotion_analyzer_project\\RawData2\\OAF_bar

### TESS Get emotion features

In [32]:
#Build list of emotions for Tess
feeling_list_tess = []

#'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad' and 'surprised' emotion classes respectively. 
#E.g., 'd03.wav' is the 3rd disgust sentence.  

emotion_dic = {"a":'angry', 
               "d":'disgust', 
               "f":'fear', 
               "h":'happy', 
               "n":'neutral', 
               "sa":'sad', 
               "su":'surprised'}

for file_path in tess_list:
    file = file_path.split("\\")[-1] 
    file_name = file.split(".")[0] 
    emotion = file_name[:-2]
    feeling_list_tess.append(emotion_dic[emotion])

#Verify emotions
feeling_list_tess

KeyError: 'OAF_back_ang'

In [ ]:
#Build dataframe from array
labels_tess = pd.DataFrame(feeling_list_tess)

In [ ]:
#Check results
labels_tess.head()

In [ ]:
#Rename column to emotion
labels_tess = labels_tess.rename({0: 'emotion'}, axis=1)

In [ ]:
#Check shape
labels_tess.shape

In [ ]:
#Check results
labels_tess.head()

In [ ]:
#Check emotion size
labels_tess_total = pd.DataFrame(labels_tess.groupby(['emotion']).size())
labels_tess_total

### TESS Get audio features

In [ ]:
rawdata_tess = pd.DataFrame(columns=['feature'])
bookmark=0

for y in tess_list:
    #Get audio features
    X, sample_rate = librosa.load(y, 
                                  res_type = res_type_s,
                                  duration = duration_s,
                                  sr = sample_rate_s,
                                  offset=offset_s)
    
    #Get MFFC features
    mfccs = librosa.feature.mfcc(   y=X, 
                                    sr = mfcc_sample_rate, 
                                    n_mfcc = n_mfcc)
    #Get MFFCs average features
    mfccs_mean = np.mean(    mfccs, 
                             axis = axis_mfcc)
    feature = mfccs_mean
    rawdata_tess.loc[bookmark] = [feature]
    bookmark=bookmark+1

In [ ]:
#Verify Tess features shape
rawdata_tess.shape

In [ ]:
#Check that there are no nan values
rawdata_tess.isnull().values.any()

In [ ]:
#Get sample data
rawdata_tess.head()

In [ ]:
#Build list
rawdata_tess_final = pd.DataFrame(rawdata_tess['feature'].values.tolist())

In [ ]:
#Check dataframe
rawdata_tess_final

## Join TESS features with targets

In [ ]:
#Concat both feature table and target table
newdf_tess = pd.concat([rawdata_tess_final,labels_tess], axis=1)
newdf_tess

In [ ]:
newdf_tess = newdf_tess.rename(index=str, columns={"0": "label"})

In [ ]:
#Verify table shape
newdf_tess.shape

In [ ]:
#Get dataframe sample data
newdf_tess.head()

In [ ]:
#Drop nan values
newdf_tess.dropna(inplace=True)
newdf_tess.shape

In [ ]:
#Shuffle rows
newdf_tess = shuffle(newdf_tess)
newdf_tess.head(10)

In [ ]:
#Verify there are no nan values
newdf_tess.isnull().values.any()

In [ ]:
#Check shape
newdf_tess.shape

In [ ]:
# See number of emotions
newdf_tess[newdf_tess.columns[-1]].nunique()

In [ ]:
#Move dataframe into separate file
newdf_tess.to_csv('emotion_capstone_final_tess_dataframe_diego_rios.csv')

# Join RAVDESS + TESS dataframes

In [ ]:
newdf_ravdess.columns

In [ ]:
newdf_tess.columns

In [ ]:
frames = [newdf_ravdess,newdf_tess]

final_dataframe = pd.concat(frames, ignore_index=True)
final_dataframe.shape

In [ ]:
#Check new and final dataframe
final_dataframe

In [ ]:
#Move dataframe into separate file
final_dataframe.to_csv('emotion_capstone_final_dataframe_diego_rios.csv')

# Dividing the data into test and train

In [ ]:
#Split features from targets
X = final_dataframe.iloc[:,:-1]

#Split targets
y = final_dataframe.iloc[:,-1]

In [ ]:
#Get sample of target
y

In [ ]:
#Get sample of features
X

In [ ]:
from sklearn.model_selection import train_test_split

#Split train & test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1)

# Check out the data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
#Check unique values for y_test
y_test.unique()

In [ ]:
#Check unique values for y_train
y_train.unique()

In [ ]:
#Label Encoding
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

#Encode emotion labels into numbers
y_train_lb = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_lb = np_utils.to_categorical(lb.fit_transform(y_test))

# Check out the data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train_lb.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test_lb.shape}')

In [ ]:
#Check encoding
np.unique(y_train_lb, axis=0)

In [ ]:
# range of x values
print(f'X range: {X_train.min()}-{X_train.max()}')
# y unique values
print(f'y values: {np.unique(y_train_lb)}')
num_classes = len(np.unique(y_train_lb))
print(f'Number of classes: {num_classes}')

## Build list of labels to build confusion matrix after model

In [ ]:
#Check encoding labels
lb.classes_

In [ ]:
#Build new lists of encoding labels
y_labels_encoded = {}
for i, label in enumerate(lb.classes_):
    y_labels_encoded[i] = label
    
y_labels_encoded

# Scale data for analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
#Normalize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scalled = scaler.transform(X_train)
X_test_scalled = scaler.transform(X_test)

# Build Model - Random Forest

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#fitting the DT
DT_model_one = DecisionTreeClassifier()
DT_model_one.fit(X_train, y_train_lb)

#Getting the score
print(f"The classification accuracy is: {DT_model_one.score(X_train, y_train_lb)}")
print(f"The classification accuracy is: {DT_model_one.score(X_test, y_test_lb)}")

# Build model - Convolution Neural Network

### Change dimensions for CNN model

In [ ]:
#Add dimension for CNN
x_traincnn = np.expand_dims(X_train_scalled, axis=2)
x_testcnn = np.expand_dims(X_test_scalled, axis=2)

#Check shapes of dataframes
print(x_traincnn.shape)
print(x_testcnn.shape)

In [ ]:
#Import packages for CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv1D 
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, BatchNormalization, Flatten, MaxPooling2D

In [ ]:
#Build sequential CNN
CNN_model = Sequential()

#Build first layer
CNN_model.add(Conv1D(16, 5,padding='same',
                 input_shape=(40, 1), activation='relu'))

#Build second layer
CNN_model.add(Conv1D(32, 5,padding='same',activation='relu'))

#Build third layer
CNN_model.add(Conv1D(64, 5,padding='same',activation='relu'))

#Build forth layer
CNN_model.add(Conv1D(128, 5,padding='same',activation='relu'))

#Add dropout
CNN_model.add(Dropout(0.1))

#Flatten 
CNN_model.add(Flatten())

CNN_model.add(Dense(128, activation ='relu'))
CNN_model.add(Dropout(0.1))
CNN_model.add(Dense(64, activation ='relu'))
CNN_model.add(Dense(8, activation='softmax'))

In [ ]:
#Look at CNN model summary
CNN_model.summary()

In [ ]:
from keras.utils import plot_model

# Save an image of the model's architecture to a file
plot_model(CNN_model, to_file='Feed Forward NN.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Compile the model with the desired loss function, optimizer, and metric to optimize
CNN_model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint 

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)

#Model fit
cnn_results = CNN_model.fit(x_traincnn, y_train_lb,
              batch_size = 64,
              epochs = 25,
              verbose = 1,
              validation_data = (x_testcnn, y_test_lb))

In [ ]:
#Plot model accuracy over ephocs
plt.plot(cnn_results.history['acc'])
plt.plot(cnn_results.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Calculate pre-training accuracy 
score = CNN_model.evaluate(x_testcnn, y_test_lb, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

In [ ]:
# Evaluating the model on the training and testing set
score = CNN_model.evaluate(x_traincnn, y_train_lb, verbose=0)
print("Training Accuracy: ", score[1])

score = CNN_model.evaluate(x_testcnn, y_test_lb, verbose=0)
print("Testing Accuracy: ", score[1])

# Add Confusion Matrix

In [ ]:
#Get predictions from model
y_test_predictions = CNN_model.predict_classes(x_testcnn)
y_test_predictions

In [ ]:
y_test

In [ ]:
#Get labels for emotions
y_labels_encoded

In [ ]:
# Change predictions to emotions in order to compare
y_test_predictions_labels =[]

#Go through each prediction and append to new list
for e in range(len(y_test_predictions)):
    y_test_predictions_labels.append(y_labels_encoded[y_test_predictions[e]])
    
#Build array of predictions
y_test_predictions_labels = np.array(y_test_predictions_labels)
y_test_predictions_labels

In [ ]:
from sklearn.metrics import confusion_matrix

#Build confusion matrix and see results
confusion_matrix = confusion_matrix(y_test, y_test_predictions_labels)
confusion_matrix

In [ ]:
#See confusion matrix shape
confusion_matrix.shape

In [ ]:
#Turn al correct answers into 0 to visualize errors better
for i in range(confusion_matrix.shape[0]):
    for j in range(confusion_matrix.shape[1]):
        if i == j:
            confusion_matrix[i,j] = 0

# See results
confusion_matrix

In [ ]:
#Add labels to confusion matrix
confusion_matrix = pd.DataFrame(confusion_matrix, columns=list(y_labels_encoded.values()), index=list(y_labels_encoded.values()))

print("The rows represents the true values or observations")
print("The columns respresent the model's predictions")

#Print confusion matrix results
confusion_matrix

In [ ]:
#Plot confusion matrix with results
ax = sns.heatmap(confusion_matrix, annot=True)

# Save CNN Model

In [ ]:
from keras.models import model_from_json
# serialize model to json
json_model = CNN_model.to_json()
#save the model architecture to JSON file
with open('capstone_project_emotion_detection_final_version.json', 'w') as json_file:
    json_file.write(json_model)
#saving the weights of the model
CNN_model.save_weights('capstone_project_emotion_detection_final_version.h5')
#Model loss and accuracy
print("Saved model to disk")

In [ ]:
# load json and create model
json_file = open('capstone_project_emotion_detection_final_version.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("capstone_project_emotion_detection_final_version.h5")
print("Loaded model from disk")

In [ ]:
from keras.initializers import glorot_uniform
#Reading the model from JSON file
with open('capstone_project_emotion_detection_final_version.json', 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model_load = keras.models.model_from_json(json_savedModel)
model_load.summary()